In [1]:
%load_ext autoreload
%autoreload 2
from typing import *

In [2]:
from utils import load_input

# Part 1

The high-speed train leaves the forest and quickly carries you south. You can even see a desert in the distance! Since you have some spare time, you might as well see if there was anything interesting in the image the Mythical Information Bureau satellite captured.

After decoding the satellite messages, you discover that the data actually contains many small images created by the satellite's camera array. The camera array consists of many cameras; rather than produce a single square image, they produce many smaller square image tiles that need to be reassembled back into a single image.

Each camera in the camera array returns a single monochrome image tile with a random unique ID number. The tiles (your puzzle input) arrived in a random order.

Worse yet, the camera array appears to be malfunctioning: each image tile has been rotated and flipped to a random orientation. Your first task is to reassemble the original image by orienting the tiles so they fit together.

To show how the tiles should be reassembled, each tile's image data includes a border that should line up exactly with its adjacent tiles. All tiles have this border, and the border lines up exactly when the tiles are both oriented correctly. Tiles at the edge of the image also have this border, but the outermost edges won't line up with any other tiles.

By rotating, flipping, and rearranging them, you can find a square arrangement that causes all adjacent borders to line up:

To check that you've assembled the image correctly, multiply the IDs of the four corner tiles together. If you do this with the assembled tiles from the example above, you get 1951 * 3079 * 2971 * 1171 = 20899048083289.

Assemble the tiles into an image. What do you get if you multiply together the IDs of the four corner tiles?



In [3]:
raw_data = load_input(20, splitlines=False)

In [4]:
raw_tiles = raw_data.strip().split("\n\n")

In [212]:
from __future__ import annotations
from functools import cached_property
from collections import namedtuple

Neighbor = namedtuple("Neighbor", "tile_id flipped")

class Tile:
    def __init__(self, tile_id: int, grid: List[List[str]]):
        self.tile_id = tile_id
        self.grid = grid
        self.neighbors = dict()

    @property
    def top(self) -> Tuple[str]:
        return self.grid[0]
    
    @property
    def bottom(self) -> Tuple[str]:
        # Note - this is backwards from its top rotation
        return self.grid[-1]
    
    @property
    def left(self) -> Tuple[str]:
        return tuple(row[0] for row in self.grid)
    
    @property
    def right(self) -> Tulple[str]:
        return tuple(row[-1] for row in self.grid)

    def rotate_clockwise(self):
        self.grid = list(zip(*reversed(self.grid)))
        
    def rotate_counterclockwise(self):
        self.grid = list(zip(*(reversed(row) for row in self.grid)))
    
    def flip(self):
        self.grid = [tuple(reversed(r)) for r in self.grid]

    def all_rotations(self) -> Iterator[Tile]:
        """
        all possible states of this tile
        """
        for _ in range(2):
            for _ in range(4):
                yield self
                self.rotate_clockwise()
            self.flip()
    
    @cached_property
    def sides(self):
        out = set()
        for _ in range(4):
            out.add(tuple(self.grid[0]))
            self.rotate_clockwise()
        return out
    
    @cached_property
    def flipped_sides(self):
        out = set()
        for _ in range(4):
            out.add(tuple(reversed(self.grid[0])))
            self.rotate_clockwise()
        return out
        
    @classmethod
    def from_raw(cls, s: str) -> Tile:
        header, tail = s.split("\n", 1)
        tile_id = int(header[5:9])
        grid = [list(row) for row in tail.splitlines()]
        return cls(tile_id, grid)
        

In [213]:
from itertools import combinations

def generate_tiles(raw_tiles) -> Tuple[List[Tile], Dict[int, Tile]]:
    tiles = [Tile.from_raw(tile) for tile in raw_tiles]
    tile_id_to_tile = {tile.tile_id:tile for tile in tiles}
    
    for left, right in combinations(tiles, 2):
        # I think this logic needs to switch because matched sides are mirror images of eachother
        # so a commmonality is a flip
        common_sides = left.sides & right.flipped_sides # sides that match will be mirror images of eachother
        for side in common_sides:
            left.neighbors[side] = Neighbor(right.tile_id, False)
            right.neighbors[side] = Neighbor(left.tile_id, False)

        common_flipped_sides = left.sides & right.sides
        for side in common_flipped_sides:
            left.neighbors[side] = Neighbor(right.tile_id, True)
            right.neighbors[side] = Neighbor(left.tile_id, True)
            
    
    return tiles, tile_id_to_tile

In [214]:
tiles, tile_id_to_tile = generate_tiles(raw_tiles)

In [215]:
import math
math.prod([tile.tile_id for tile in tiles if len(tile.neighbors) == 2])

140656720229539

## Part 2

Now, you're ready to check the image for sea monsters.

The borders of each tile are not part of the actual image; start by removing them.

In the example above, the tiles become:
```
.#.#..#. ##...#.# #..#####
###....# .#....#. .#......
##.##.## #.#.#..# #####...
###.#### #...#.## ###.#..#
##.#.... #.##.### #...#.##
...##### ###.#... .#####.#
....#..# ...##..# .#.###..
.####... #..#.... .#......

#..#.##. .#..###. #.##....
#.####.. #.####.# .#.###..
###.#.#. ..#.#### ##.#..##
#.####.. ..##..## ######.#
##..##.# ...#...# .#.#.#..
...#..#. .#.#.##. .###.###
.#.#.... #.##.#.. .###.##.
###.#... #..#.##. ######..

.#.#.### .##.##.# ..#.##..
.####.## #.#...## #.#..#.#
..#.#..# ..#.#.#. ####.###
#..####. ..#.#.#. ###.###.
#####..# ####...# ##....##
#.##..#. .#...#.. ####...#
.#.###.. ##..##.. ####.##.
...###.. .##...#. ..#..###
```
Remove the gaps to form the actual image:
```
.#.#..#.##...#.##..#####
###....#.#....#..#......
##.##.###.#.#..######...
###.#####...#.#####.#..#
##.#....#.##.####...#.##
...########.#....#####.#
....#..#...##..#.#.###..
.####...#..#.....#......
#..#.##..#..###.#.##....
#.####..#.####.#.#.###..
###.#.#...#.######.#..##
#.####....##..########.#
##..##.#...#...#.#.#.#..
...#..#..#.#.##..###.###
.#.#....#.##.#...###.##.
###.#...#..#.##.######..
.#.#.###.##.##.#..#.##..
.####.###.#...###.#..#.#
..#.#..#..#.#.#.####.###
#..####...#.#.#.###.###.
#####..#####...###....##
#.##..#..#...#..####...#
.#.###..##..##..####.##.
...###...##...#...#..###
```
Now, you're ready to search for sea monsters! Because your image is monochrome, a sea monster will look like this:
```
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
```
When looking for this pattern in the image, the spaces can be anything; only the # need to match. Also, you might need to rotate or flip your image before it's oriented correctly to find sea monsters. In the above image, after flipping and rotating it to the appropriate orientation, there are two sea monsters (marked with O):
```
.####...#####..#...###..
#####..#..#.#.####..#.#.
.#.#...#.###...#.##.O#..
#.O.##.OO#.#.OO.##.OOO##
..#O.#O#.O##O..O.#O##.##
...#.#..##.##...#..#..##
#.##.#..#.#..#..##.#.#..
.###.##.....#...###.#...
#.####.#.#....##.#..#.#.
##...#..#....#..#...####
..#.##...###..#.#####..#
....#.##.#.#####....#...
..##.##.###.....#.##..#.
#...#...###..####....##.
.#.##...#.##.#.#.###...#
#.###.#..####...##..#...
#.###...#.##...#.##O###.
.O##.#OO.###OO##..OOO##.
..O#.O..O..O.#O##O##.###
#.#..##.########..#..##.
#.#####..#.#...##..#....
#....##..#.#########..##
#...#.....#..##...###.##
#..###....##.#...##.##.#
```
Determine how rough the waters are in the sea monsters' habitat by counting the number of # that are not part of a sea monster. In the above example, the habitat's water roughness is 273.

How many # are not part of a sea monster?



Note: the below was taken from https://github.com/joelgrus/advent2020/blob/master/advent2020/day20.py after trying many many things that did not work. _shrug_

In [255]:
Edge = str

class Edges(NamedTuple):
    top: Edge
    bottom: Edge
    left: Edge
    right: Edge

Pixels = List[List[str]]

class Tile(NamedTuple):
    tile_id: int
    pixels: Pixels

    def rotate(self, n: int) -> Tile:
        """
        Rotate the tile clockwise n times
        and return a new Tile object
        """
        pixels = self.pixels
        for _ in range(n):
            rotated = []
            for c in range(len(pixels[0])):
                rotated.append([row[c] for row in reversed(pixels)])
            pixels = rotated
        return self._replace(pixels=pixels)

    def flip_horizontal(self, do: bool = False) -> Tile:
        """
        Flip the tile horizontally and return a new tile object
        """
        pixels = [list(reversed(row)) for row in self.pixels] if do else self.pixels
        return self._replace(pixels=pixels)

    def flip_vertical(self, do: bool = False) -> Tile:
        """
        Flip the tile vertically and return a new tile object
        """
        pixels = list(reversed(self.pixels)) if do else self.pixels
        return self._replace(pixels=pixels)

    def all_rotations(self) -> Iterator[Tile]:
        """
        Return the 8 tiles I can get from this one 
        by doing rotations and flips
        """
        for flip_h in [True, False]:
            for rot in [0, 1, 2, 3]:
                yield (self
                    .flip_horizontal(flip_h)
                    .rotate(rot)
                )
                        

    def show(self) -> None:
        for row in self.pixels:
            print(''.join(row))

    @property
    def top(self) -> str:
        return ''.join(self.pixels[0])

    @property
    def bottom(self) -> str:
        return ''.join(self.pixels[-1])

    @property
    def left(self) -> str:
        return ''.join([row[0] for row in self.pixels])

    @property
    def right(self) -> str:
        return ''.join([row[-1] for row in self.pixels])

    def edges(
        self,
        reverse: bool = False
    ) -> Edges:
        """
        Returns the edges of the tile as strings.
        If reverse == True, rotates the tile by 180 degrees first,
        which results in all the edges being in the opposite direction
        """
        if reverse:
            return self.rotate(2).edges()
        return Edges(
            top=self.top, bottom=self.bottom, right=self.right, left=self.left
        )

    @staticmethod
    def parse(raw_tile: str) -> Tile:
        lines = raw_tile.split("\n")
        tile_id = int(lines[0].split()[-1][:-1])
        pixels = [list(line) for line in lines[1:]]
        return Tile(tile_id, pixels)

def make_tiles(raw: str) -> List[Tile]:
    tiles_raw = raw.split("\n\n")
    return [Tile.parse(tile_raw) for tile_raw in tiles_raw]


def find_corners(tiles: List[Tile]) -> List[Tile]:
    """
    Return corners oriented so that
    they would be the top left corner
    """
    # count up all the edges / reverse edges that occur
    # for example, if a tile had the top edge "ABCD",
    # we would count "ABCD" once and also "DCBA" once
    edge_counts = Counter(
        edge 
        for tile in tiles 
        for reverse in [True, False]
        for edge in tile.edges(reverse)
    )

    corners = []

    for tile in tiles:
        sides_with_no_matches = 0
        for edge in tile.edges():
            if edge_counts[edge] == 1 and edge_counts[edge[::-1]] == 1:
                sides_with_no_matches += 1

        if sides_with_no_matches == 2:
            # rotate to get corner edges at top and left
            for rot in [0, 1, 2, 3]:
                tile = tile.rotate(rot)
                edges = tile.edges()

                if edge_counts[edges.left] == 1 and edge_counts[edges.top] == 1:
                    corners.append(tile)
                    break

    return corners


Assembly = List[List[Optional[Tile]]] 

class Constraint(NamedTuple):
    """
    Says that the tile at location (i, j)
    must have sides that match the specified
    top / bottom / left / right
    """
    i: int
    j: int
    top: Optional[str] = None
    bottom: Optional[str] = None
    left: Optional[str] = None
    right: Optional[str] = None

    def satisfied_by(self, tile: Tile) -> bool:
        """
        Does the tile satisfy this constraint
        """
        if self.top and tile.top != self.top:
            return False
        if self.bottom and tile.bottom != self.bottom:
            return False
        if self.left and tile.left != self.left:
            return False
        if self.right and tile.right != self.right:
            return False
        return True
        
    @property
    def num_constraints(self) -> int:
        return (
            (self.top is not None) +
            (self.bottom is not None) + 
            (self.left is not None) + 
            (self.right is not None)
        )

def find_constraints(assembly: Assembly) -> Iterator[Constraint]:
    """
    Create constraints from a (partially filled in) Assembly.
    No constraints for already-filled-in tiles or unconstrained locations.
    """
    n = len(assembly)

    for i, row in enumerate(assembly):
        for j, tile in enumerate(row):
            # already have a tile here
            if assembly[i][j]:
                continue
            constraints: Dict[str, str] = {}
            if i > 0 and (nbr := assembly[i-1][j]):
                constraints["top"] = nbr.bottom
            if i < n-1 and (nbr := assembly[i+1][j]):
                constraints["bottom"] = nbr.top 
            if j > 0 and (nbr := assembly[i][j-1]):
                constraints["left"] = nbr.right
            if j < n-1 and (nbr := assembly[i][j+1]):
                constraints["right"] = nbr.left

            if constraints:
                yield Constraint(i, j, **constraints)


def assemble_image(tiles: List[Tile]) -> Assembly:
    """
    Take the tiles and figure out how to stick them together
    """
    num_tiles = len(tiles)
    side_length = int(math.sqrt(num_tiles))
    corners = find_corners(tiles)

    # Pick a corner, any corner
    tile = corners[0]

    # Create an empty assembly
    assembly: Assembly = [[None for _ in range(side_length)] for _ in range(side_length)]

    # Put this corner tile in the top left
    assembly[0][0] = tile

    # Keep track of which tiles I've already placed
    placed: Dict[int, Tuple[int, int]] = {tile.tile_id: (0, 0)}

    # Repeat until all tiles have been placed
    while len(placed) < num_tiles:
        # Just care about unplaced tiles
        tiles = [t for t in tiles if t.tile_id not in placed]

        # Find the constraints based on all the tiles placed so far
        # and order them by descending # of constraints
        constraints = list(find_constraints(assembly))
        constraints.sort(key=lambda c: c.num_constraints, reverse=True)

        # Did I find a tile to add, so we can break out of inner loops
        found_one = False

        # Try constraints one at a time and see if we can find a tile
        # that satisfies them
        for constraint in constraints:
            for tile in tiles:
                # try all rotations for this tile, to see if any satisfies this constraint
                for rot in tile.all_rotations():
                    if constraint.satisfied_by(rot):
                        # place this rotation (which is a tile) at i, j
                        assembly[constraint.i][constraint.j] = rot 
                        placed[rot.tile_id] = (constraint.i, constraint.j)
                        found_one = True
                        break
                if found_one:
                    break
            if found_one:
                break

    return assembly

def glue(assembly: Assembly) -> Pixels:
    """
    Glue together the Tiles into a single grid of pixels,
    removing the edges of each tile
    """
    N = len(assembly)
    n = len(assembly[0][0].pixels)
    nout = (n - 2) * N
    glued = [['' for _ in range(nout)] for _ in range(nout)]
    for i, row in enumerate(assembly):
        for j, tile in enumerate(row):
            cropped = [line[1:-1] for line in tile.pixels[1:-1]]
            for ii, crow in enumerate(cropped):
                for jj, pixel in enumerate(crow):
                    glued[i * (n-2) + ii][j * (n-2) + jj] = pixel

    return glued

SEA_MONSTER_RAW = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #"""   

# offsets for a sea monster
SEA_MONSTER = [
    (i, j) 
    for i, row in enumerate(SEA_MONSTER_RAW.split("\n"))
    for j, c in enumerate(row)
    if c == '#']

def find_sea_monsters(pixels: Pixels) -> Iterator[Tuple[int, int]]:
    """
    Return the indices of the top left corner of each sea monster
    """
    for i, row in enumerate(pixels):
        for j, c in enumerate(row):
            try:
                if all(pixels[i + di][j + dj] == '#' for di, dj in SEA_MONSTER):
                    yield (i, j)
            except IndexError:
                continue

def roughness(glued: Pixels) -> int:
    """
    Count the #s that are not part of a sea monster
    """
    # put the pixels in a Tile so we can use Tile methods
    tile = Tile(0, glued)

    # for each of the 8 rotation/flips, find the list of sea monster top lefts
    finds = [(t, list(find_sea_monsters(t.pixels))) for t in tile.all_rotations()]

    # only keep the ones that had sea monsters
    finds = [(t, sm) for t, sm in finds if sm]

    # hopefully only one of them had sea monsters
    assert len(finds) == 1

    # and that's our tile (and sea monster locations)
    t, sms = finds[0]

    # now we can computer all pixels that are showing a sea monster
    sea_monster_pixels = {(i + di, j + dj)
                          for i, j in sms
                          for di, dj in SEA_MONSTER}  

    # and count all the '#'s that are not sea monster pixels
    return sum(c == '#' and (i, j) not in sea_monster_pixels
               for i, row in enumerate(t.pixels)
               for j, c in enumerate(row))



In [260]:
tiles = make_tiles(raw_data.strip())

In [261]:
corners = find_corners(tiles)
assert len(corners) == 4
print(math.prod(tile.tile_id for tile in corners))
images = assemble_image(tiles)
glued = glue(images)
print(roughness(glued))


140656720229539
1885


## Stuff that Just did not work

In [154]:
from IPython.core.debugger import set_trace


In [183]:
def get_aligned_column(top_most_tile: Tile) -> List[Tile]:
    aligned_tiles = []
    current_tile = top_most_tile
    while True:
        aligned_tiles.append(current_tile)
        #set_trace()
        if (bottom:=current_tile.bottom) in current_tile.neighbors:
            next_id, _ = current_tile.neighbors[bottom]
            next_flipped = False
        elif (bottom:=tuple(reversed(current_tile.bottom))) in current_tile.neighbors:
            next_id, _ = current_tile.neighbors[bottom]
            next_flipped = True
        else:
            break
        
        next_tile = tile_id_to_tile[next_id]
        
        if next_flipped:
            next_tile.flip()
        
        rotation_count = 0
        while next_tile.top != bottom:
            if rotation_count == 4:
                # this is dumb but the logic for keeping track of "should flip" is complicated
                next_tile.flip() 
            next_tile.rotate_clockwise()
            rotation_count += 1
            if rotation_count > 8:
                raise RuntimeError("something is wrong")
            
        current_tile = next_tile
        
    return aligned_tiles

In [184]:
first_col = get_aligned_column(top_left_tile)

In [194]:
from itertools import chain 

def build_row(left_most_tile: Tile) -> List[Tuple[str]]:
    """
    We go through and align all tiles, then zip them, chaining together
    """
    aligned_tiles = []
    current_tile = left_most_tile
    while True:
        aligned_tiles.append(current_tile)
        
        if (right:=current_tile.right) in current_tile.neighbors:
            next_id, _ = current_tile.neighbors[right]
            next_flipped = False
        elif (right:=tuple(reversed(current_tile.right))) in current_tile.neighbors:
            next_id, _ = current_tile.neighbors[right]
            next_flipped = True
        else:
            break

        
        next_tile = tile_id_to_tile[next_id]

        if next_flipped:
            next_tile.flip()

        rotation_count = 0
        while next_tile.left != right:
            if rotation_count == 4:
                # this is dumb but the logic for keeping track of "should flip" is complicated
                next_tile.flip() 
            next_tile.rotate_clockwise()
            rotation_count += 1
            if rotation_count > 8:
                raise RuntimeError("something is wrong")
        
        current_tile = next_tile
    assert len(aligned_tiles) == 12
    return aligned_tiles
#     aligned_grids = [tile.grid for tile in aligned_tiles]
#     return [tuple(chain(*rows)) for rows in zip(*aligned_grids)]
    

In [195]:
rows = list(map(build_row, first_col))

AssertionError: 

In [196]:
%debug

> <ipython-input-194-9d282ba4ea3b>(38)build_row()
     36 
     37         current_tile = next_tile
---> 38     assert len(aligned_tiles) == 12
     39     return aligned_tiles
     40 #     aligned_grids = [tile.grid for tile in aligned_tiles]



ipdb>  current_tile


ipdb>  right


('.', '.', '.', '#', '.', '.', '#', '.', '#', '.')


ipdb>  current_tile.neighbors


{('#', '.', '.', '#', '#', '.', '.', '.', '.', '.'): Neighbor(tile_id=1913, flipped=True), ('#', '.', '#', '.', '#', '#', '#', '.', '#', '#'): Neighbor(tile_id=2239, flipped=False), ('#', '.', '.', '#', '#', '#', '#', '#', '#', '.'): Neighbor(tile_id=1823, flipped=True)}


ipdb>  current_tile.left


('#', '.', '#', '.', '#', '#', '#', '.', '#', '#')


ipdb>  q


In [160]:
first_row = build_row(top_left_tile)

In [164]:
[len(x) for x in list(map(get_aligned_column,first_row))]

[12, 12, 1, 12, 1, 12, 12, 1, 1, 1, 1, 1]

In [169]:
first_row[1].grid

[('#', '.', '#', '#', '#', '.', '.', '.', '#', '.'),
 ('.', '.', '.', '.', '.', '.', '.', '.', '#', '#'),
 ('#', '.', '.', '.', '.', '.', '.', '.', '.', '#'),
 ('#', '#', '.', '#', '#', '.', '.', '.', '.', '#'),
 ('#', '.', '.', '.', '.', '.', '.', '#', '.', '.'),
 ('#', '.', '.', '#', '.', '.', '#', '.', '.', '#'),
 ('.', '.', '#', '.', '.', '#', '#', '#', '#', '#'),
 ('.', '.', '.', '.', '.', '.', '.', '.', '#', '#'),
 ('.', '.', '.', '#', '#', '.', '.', '.', '.', '#'),
 ('.', '#', '.', '#', '#', '#', '.', '.', '#', '#')]

In [167]:
first_row[2].grid

[('#', '.', '.', '.', '#', '#', '#', '#', '#', '#'),
 ('#', '#', '.', '.', '#', '.', '.', '.', '.', '#'),
 ('#', '.', '.', '#', '#', '.', '#', '#', '#', '.'),
 ('#', '.', '.', '.', '#', '#', '#', '#', '.', '#'),
 ('#', '.', '.', '#', '.', '.', '.', '.', '.', '.'),
 ('.', '#', '.', '.', '.', '.', '.', '.', '#', '#'),
 ('#', '.', '.', '#', '#', '.', '#', '.', '.', '#'),
 ('#', '.', '.', '.', '.', '.', '.', '.', '#', '.'),
 ('#', '.', '.', '.', '.', '.', '.', '#', '#', '#'),
 ('.', '#', '#', '#', '#', '.', '.', '#', '.', '.')]

In [168]:
first_row[2].neighbors

{('.',
  '#',
  '.',
  '#',
  '#',
  '.',
  '#',
  '.',
  '#',
  '#'): Neighbor(tile_id=1321, flipped=True),
 ('#',
  '#',
  '#',
  '#',
  '#',
  '.',
  '#',
  '#',
  '#',
  '.'): Neighbor(tile_id=2251, flipped=False),
 ('#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '.',
  '.',
  '.',
  '#'): Neighbor(tile_id=3701, flipped=False)}